# debug precompute

## Load data

In [1]:
from ipywidgets import interact
import os 
import nibabel as nib
import plotui
import numpy as np
root='alg_out/'

@interact
def selectData(dataset=['Task001_LiverTumor','Task002_Pancreas','Task003_Multiorgan']):
    @interact
    def fileviewer(file=sorted(os.listdir(f'{root}/{dataset}/GroundTruth'))):
        global gt,voxelsize
        gt_nib = nib.load(f'{root}/{dataset}/GroundTruth/{file}')
        gt=gt_nib.dataobj[...]
        from nibabel.affines import voxel_sizes
        voxelsize=voxel_sizes(gt_nib.affine)
        @interact
        def fileviewer2(typ=['CE','Dice','DiceCE','DiceFocal','DiceTopK10','TopK10']):
                global pr
                pr_nib = nib.load(f'{root}/{dataset}/{typ}/{file}')
                if (voxelsize!=voxel_sizes(pr_nib.affine)).any():
                    raise Error('voxel sizes are different')
                pr=pr_nib.dataobj[...]
                print(f'dataset {dataset} predict {typ} loaded....')
                

            

interactive(children=(Dropdown(description='dataset', options=('Task001_LiverTumor', 'Task002_Pancreas', 'Task…

In [8]:
import myutils
gt2=myutils.array_trim(gt)

In [9]:
import importlib
import metric
importlib.reload(metric)

mme=metric.MME(gt2.max()+1,debug={'show_precompute':0})
helper=mme.precompute(gt2,spacing=voxelsize)

0
28.972s [1 * 28.972s]  class 0
├── 8.49s [1 * 8.49s]  component0
│   ├── 519.094ms [1 * 519.094ms]  boundary
│   ├── 4.634s [1 * 4.634s]  distances
│   └── 3.319s [1 * 3.319s]  skeleton
└── 20.409s [1 * 20.409s]  component1
    ├── 4.49s [1 * 4.49s]  distances
    └── 15.454s [1 * 15.454s]  skeleton

1
1.958m [1 * 1.958m]  class 1
├── 20.482s [1 * 20.482s]  component0
│   ├── 4.601s [1 * 4.601s]  distances
│   └── 15.419s [1 * 15.419s]  skeleton
├── 7.06s [1 * 7.06s]  component1
│   ├── 3.849s [1 * 3.849s]  distances
│   └── 2.739s [1 * 2.739s]  skeleton
├── 6.645s [1 * 6.645s]  component2
│   ├── 3.586s [1 * 3.586s]  distances
│   └── 2.604s [1 * 2.604s]  skeleton
├── 5.696s [1 * 5.696s]  component3
│   ├── 3.43s [1 * 3.43s]  distances
│   └── 1.78s [1 * 1.78s]  skeleton
├── 7.382s [1 * 7.382s]  component4
│   ├── 3.82s [1 * 3.82s]  distances
│   └── 3.106s [1 * 3.106s]  skeleton
├── 6.17s [1 * 6.17s]  component5
│   ├── 3.564s [1 * 3.564s]  distances
│   └── 2.148s [1 * 2.148s]  sk

In [3]:
import matplotlib.pyplot as plt

shape=gt[:,:,gt.sum(axis=0).sum(axis=0).argmax()].astype(np.uint8)
shape=gt
# shape=np.zeros((200,200),np.uint8)
# shape[10:90,10:50]=1

# shape[10:90,60:100]=1
import cc3d
marker=cc3d.connected_components(shape).astype(np.int8)
# marker=scipy.ndimage.find_objects(shape.astype(np.int8),3)
# plt.imshow(marker)


In [15]:
a=np.zeros(5)
a[1]=1

np.unique(a).delete(0)

AttributeError: 'numpy.ndarray' object has no attribute 'delete'

In [5]:
import scipy
from auto_profiler import Timer
# with Timer.instance('dis'):
#     dst=scipy.ndimage.distance_transform_edt(shape>0)#+scipy.ndimage.distance_transform_edt(shape)
ddd=np.ones(marker.shape,np.uint8)

mark=-(marker)
inp=dst
with Timer.instance('watershed'):
    res=scipy.ndimage.watershed_ift(inp.astype(np.uint8), mark.astype(np.int8), structure=None, output=None)
res
# plt.imshow(-dst)

2.608m [2 * 1.304m]  watershed



array([[[-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -9, -9, -9]],

       [[-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -9, -9, -9]],

       [[-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -9, -9, -9],
        [-9, -9, -9, ..., -9, -9, -9]],

       ...,

       [[-5, -5, -5, ..., -5, -5, -5],
        [-5, -5, -5, ..., -5, -5, -5],
        [-5, -5, -5, ..., -5, -5, -5],
        ...,
        [-5, -5, -5, ..., -5, -5, -5],
        [-5, -5, -5, ..., -5, -5, -5],
        [-5, -5, -5, ..., -5, -5

In [7]:
res.min()

-17

In [4]:
mark


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [119]:
marker.max()

2